In [1]:
import pandas as pd
import numpy as np

from statsmodels.stats.inter_rater import fleiss_kappa

# 0. Get data

In [2]:
# get sample 1 
sample_1 = pd.read_csv("./mturk/webisa_0_sample_500_mturk_output.csv",
                      sep=",")
sample_1.head()

,HITId,HITTypeId,Title,Description,Keywords,Reward,CreationTime,MaxAssignments,RequesterAnnotation,AssignmentDurationInSeconds,...,Answer.95659273,Answer.97185308,Answer.97510057,Answer.98432274,Answer.98778070,Answer.99335721,Answer.99559518,Answer.99875032,Approve,Reject
0,3P4ZBJFX2VC79WRWE9UYSMR8O15WFZ,34645SSUIB3FQKQFEOCGBDB4N39X0K,"Judge ""is a"" relations - like ""paris is a city""","You should decide whether a ""is a"" relations h...","isa, judgement",$0.05,Sun Apr 02 06:27:23 PDT 2017,9,BatchId:2749188;OriginalHitTemplateId:920937332;,1200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3P4ZBJFX2VC79WRWE9UYSMR8O15WFZ,34645SSUIB3FQKQFEOCGBDB4N39X0K,"Judge ""is a"" relations - like ""paris is a city""","You should decide whether a ""is a"" relations h...","isa, judgement",$0.05,Sun Apr 02 06:27:23 PDT 2017,9,BatchId:2749188;OriginalHitTemplateId:920937332;,1200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3P4ZBJFX2VC79WRWE9UYSMR8O15WFZ,34645SSUIB3FQKQFEOCGBDB4N39X0K,"Judge ""is a"" relations - like ""paris is a city""","You should decide whether a ""is a"" relations h...","isa, judgement",$0.05,Sun Apr 02 06:27:23 PDT 2017,9,BatchId:2749188;OriginalHitTemplateId:920937332;,1200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3P4ZBJFX2VC79WRWE9UYSMR8O15WFZ,34645SSUIB3FQKQFEOCGBDB4N39X0K,"Judge ""is a"" relations - like ""paris is a city""","You should decide whether a ""is a"" relations h...","isa, judgement",$0.05,Sun Apr 02 06:27:23 PDT 2017,9,BatchId:2749188;OriginalHitTemplateId:920937332;,1200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3P4ZBJFX2VC79WRWE9UYSMR8O15WFZ,34645SSUIB3FQKQFEOCGBDB4N39X0K,"Judge ""is a"" relations - like ""paris is a city""","You should decide whether a ""is a"" relations h...","isa, judgement",$0.05,Sun Apr 02 06:27:23 PDT 2017,9,BatchId:2749188;OriginalHitTemplateId:920937332;,1200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# too many columns to handle, delete all unnecessary columns
sample_1.drop(columns=['HITId', 'HITTypeId', 'Title', 'Description',
                      'Keywords', 'Reward', 'CreationTime', 'MaxAssignments',
                      'RequesterAnnotation', 'AssignmentDurationInSeconds', 
                      'AutoApprovalDelayInSeconds', 'Expiration',
                      'NumberOfSimilarHITs', 'LifetimeInSeconds'], inplace=True)

In [4]:
# too many columns to handle, delete all unnecessary columns
sample_1.drop(columns=['AssignmentId', 'LifetimeApprovalRate',
                      'Last30DaysApprovalRate', 'Last7DaysApprovalRate',
                      'Approve', 'Reject'], inplace=True)

In [5]:
sample_1.shape

(225, 549)

In [6]:
# drop all columns with Input.r_ pattern
cols = [c for c in sample_1.columns if c[:8] != 'Input.r_'] #if c.lower()[:6] != 'input.r_']
sample_1 = sample_1[cols]

In [7]:
# drop first few columns
sample_1.drop(columns=['AssignmentStatus', 'AcceptTime',
                      'SubmitTime', 'AutoApprovalTime', 'ApprovalTime',
                      'RejectionTime', 'RequesterFeedback', 'WorkTimeInSeconds'],
             inplace=True)

In [8]:
sample_1.head()

,WorkerId,Input.id_0,Input.id_1,Input.id_2,Input.id_3,Input.id_4,Input.id_5,Input.id_6,Input.id_7,Input.id_8,...,Answer.94756223,Answer.95228370,Answer.95659273,Answer.97185308,Answer.97510057,Answer.98432274,Answer.98778070,Answer.99335721,Answer.99559518,Answer.99875032
0,APR6H3HAOE9OU,516227103,54225214,502492695,124467686,344425380,270852864,369033327,484633465,516170918,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A19IWFLUTW6KQU,516227103,54225214,502492695,124467686,344425380,270852864,369033327,484633465,516170918,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A3HIO96PK1JA03,516227103,54225214,502492695,124467686,344425380,270852864,369033327,484633465,516170918,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A31IX5RLQ7MMRY,516227103,54225214,502492695,124467686,344425380,270852864,369033327,484633465,516170918,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A1G187YBG0DVMQ,516227103,54225214,502492695,124467686,344425380,270852864,369033327,484633465,516170918,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data = []
# get the right answer per workerid and input
for index, row in sample_1.iterrows():
    
    # loop through all answerst per workerId
    for i in range(0, 20):
        
        # get 
        column_name = 'Input.id_' + str(i)
        
        # get id of question
        question_id = str(row[column_name])
        
        # get answer
        answer = 'Answer.' + question_id
        answer = row[answer]
        
        data.append([row['WorkerId'], question_id, answer])
        
        

In [10]:
# all data
all_data = pd.DataFrame(data,  columns=['Annotator', 'id', 'label'])
all_data

,Annotator,id,label
0,APR6H3HAOE9OU,516227103,yes
1,APR6H3HAOE9OU,54225214,yes
2,APR6H3HAOE9OU,502492695,yes
3,APR6H3HAOE9OU,124467686,yes
4,APR6H3HAOE9OU,344425380,yes
...,...,...,...
4495,A1G187YBG0DVMQ,178754622,no
4496,A1G187YBG0DVMQ,418781429,no
4497,A1G187YBG0DVMQ,89573263,no
4498,A1G187YBG0DVMQ,224542410,no


In [11]:
def get_data(str_folder):
    sample = pd.read_csv(str_folder,
                      sep=",")
    
    # too many columns to handle, delete all unnecessary columns
    sample.drop(columns=['HITId', 'HITTypeId', 'Title', 'Description',
                      'Keywords', 'Reward', 'CreationTime', 'MaxAssignments',
                      'RequesterAnnotation', 'AssignmentDurationInSeconds', 
                      'AutoApprovalDelayInSeconds', 'Expiration',
                      'NumberOfSimilarHITs', 'LifetimeInSeconds',
                      'AssignmentId', 'LifetimeApprovalRate',
                      'Last30DaysApprovalRate', 'Last7DaysApprovalRate',
                      'Approve', 'Reject', 
                      'AssignmentStatus', 'AcceptTime',
                      'SubmitTime', 'AutoApprovalTime', 'ApprovalTime',
                      'RejectionTime', 'RequesterFeedback', 'WorkTimeInSeconds'], inplace=True)
    
    # drop all columns with Input.r_ pattern
    cols = [c for c in sample.columns if c[:8] != 'Input.r_'] #if c.lower()[:6] != 'input.r_']
    sample = sample[cols]
    
    data = []
    # get the right answer per workerid and input
    for index, row in sample.iterrows():
    
        # loop through all answerst per workerId
        for i in range(0, 20):
        
            # get 
            column_name = 'Input.id_' + str(i)
        
            # get id of question
            question_id = str(row[column_name])
        
            # get answer
            answer = 'Answer.' + question_id
            answer = row[answer]
        
            data.append([row['WorkerId'], question_id, answer])
        
    return data

In [23]:
# loop through all folders
all_data = []

folder = "./mturk/webisa_" 
#10,20
for j in [0,1,2,3,5,10,20]:
    
    folder_name = folder + str(j) + '_sample_500_mturk_output.csv'
    print(folder_name)
    
    data = get_data(folder_name)
    
    all_data.extend(data)

/home/linda/Dokumente/Masterarbeit/6_Git/mturk/webisa_0_sample_500_mturk_output.csv
/home/linda/Dokumente/Masterarbeit/6_Git/mturk/webisa_1_sample_500_mturk_output.csv
/home/linda/Dokumente/Masterarbeit/6_Git/mturk/webisa_2_sample_500_mturk_output.csv
/home/linda/Dokumente/Masterarbeit/6_Git/mturk/webisa_3_sample_500_mturk_output.csv
/home/linda/Dokumente/Masterarbeit/6_Git/mturk/webisa_5_sample_500_mturk_output.csv
/home/linda/Dokumente/Masterarbeit/6_Git/mturk/webisa_10_sample_500_mturk_output.csv
/home/linda/Dokumente/Masterarbeit/6_Git/mturk/webisa_20_sample_500_mturk_output.csv


In [24]:
# convert to dataframe for easier indexing
all_data = pd.DataFrame(all_data, columns=['Annotator', 'id', 'label'])
all_data

,Annotator,id,label
0,APR6H3HAOE9OU,516227103,yes
1,APR6H3HAOE9OU,54225214,yes
2,APR6H3HAOE9OU,502492695,yes
3,APR6H3HAOE9OU,124467686,yes
4,APR6H3HAOE9OU,344425380,yes
...,...,...,...
31495,A19IWFLUTW6KQU,176464507,yes
31496,A19IWFLUTW6KQU,190698478,no
31497,A19IWFLUTW6KQU,305259200,no
31498,A19IWFLUTW6KQU,454099429,no


In [25]:
# show whether function worked
all_data['Key'] = all_data['Annotator'] + ' ' + all_data['id'] + ' ' + all_data['label']
all_data.Key.value_counts()

A5IG6AWG6W638 454099429 no      2
A1SP4XA17LVUQI 454099429 no     2
A19IWFLUTW6KQU 172634227 yes    2
AMA18W8F60Y2J 454099429 no      2
A1G187YBG0DVMQ 172634227 no     2
                               ..
A1SP4XA17LVUQI 34286723 no      1
A19WXS1CLVLEEX 247506053 yes    1
A340UGIDGIF4W6 146172609 no     1
A3HIO96PK1JA03 504511568 no     1
A1SP4XA17LVUQI 38023182 yes     1
Name: Key, Length: 31483, dtype: int64

In [15]:
duplicates = pd.DataFrame(all_data.Key.value_counts())
duplicates = duplicates[duplicates['Key'] > 1].reset_index()
duplicates.columns = ['Key', 'Counts']
duplicates

,Key,Counts
0,A5IG6AWG6W638 454099429 no,2
1,A1SP4XA17LVUQI 454099429 no,2
2,A19IWFLUTW6KQU 172634227 yes,2
3,AMA18W8F60Y2J 454099429 no,2
4,A1G187YBG0DVMQ 172634227 no,2
5,A6T8PXIZ6IN7Y 454099429 no,2
6,A3JP7DIFRS0ZBY 454099429 no,2
7,A1G187YBG0DVMQ 454099429 no,2
8,A1G187YBG0DVMQ 447299358 no,2
9,A6T8PXIZ6IN7Y 447299358 no,2


In [26]:
# get distinct annotators
all_data.Annotator.value_counts()

A1G187YBG0DVMQ    3500
A6T8PXIZ6IN7Y     2860
A19IWFLUTW6KQU    2160
A3JP7DIFRS0ZBY    1940
A5IG6AWG6W638     1860
                  ... 
A3DSDNBIYC78DK      20
AIXLQHGEOFYQH       20
A1UDB1L9AID4AF      20
A2PAV7JATODL20      20
A1I4CYG5YDFTYM      20
Name: Annotator, Length: 91, dtype: int64

In [17]:
# remove duplicate keys from dataframe
all_data = all_data[~all_data['Key'].isin(duplicates.Key.to_list())]
len(all_data)

31468

In [18]:
# get all data id
all_data.id.value_counts()

172634227    14
498845835    12
447299358    10
302867080     9
453014549     9
             ..
104163259     9
273128171     9
20039415      9
164409489     9
454099429     4
Name: id, Length: 3496, dtype: int64

In [19]:
# delete Keys as column
del all_data['Key']

In [22]:
# show how often the data was filled
all_data.Annotator.value_counts()

A1G187YBG0DVMQ    3492
A6T8PXIZ6IN7Y     2854
A19IWFLUTW6KQU    2154
A3JP7DIFRS0ZBY    1938
A5IG6AWG6W638     1858
                  ... 
A3DSDNBIYC78DK      20
AIXLQHGEOFYQH       20
A1UDB1L9AID4AF      20
A2PAV7JATODL20      20
A1I4CYG5YDFTYM      20
Name: Annotator, Length: 91, dtype: int64

In [20]:
# pivot the values
all_data_pivot = all_data.pivot_table(values='label',
                                     index='id',
                                     columns='Annotator',
                                     aggfunc='first')

all_data_pivot

Annotator,A10DGGOQ9N186P,A13NXSZ9D5NTHN,A19IWFLUTW6KQU,A19R04IXWA4ZLX,A19WXS1CLVLEEX,A1AJIQ428EJYST,A1ANG01DQJZCUZ,A1B22KYHU5D0NF,A1BAEFJ2OG7NJ0,A1CSQCS9NEM5GV,...,AR227RT5PZ16I,ARM2CBQGJCQ5Z,AS5P5QIEGUD5X,ASLGHXA4453BQ,ATADQXPHL10Y8,AUHRS2GFPRADF,AVHJ2SNIMGZ91,AYFAOYFBISRF3,AYJGJAIY0EXW,AZ8KL6F16ZS7G
id,,,,,,,,,,,,,,,,,,,,,
100136270,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100270049,NaN,no,no,NaN,no,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN
100324230,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100521343,yes,no,NaN,NaN,NaN,yes,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100585772,NaN,NaN,no,NaN,NaN,NaN,no,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99416385,NaN,no,yes,NaN,yes,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN
99559518,NaN,NaN,NaN,NaN,NaN,uncertain,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN
99611426,NaN,no,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN


In [ ]:
def pad_with_num_values(row)

In [ ]:
all_data_pivot